### WIP: Prep to implement DenseNet for GFW vessel classification
##### Implementation in Slim for GFW
##### ....need to get MNIST working first

In [ ]:
import tensorflow as tf 

In [ ]:
def bn_relu_conv(net, n_channels, drop, is_training=True, scope="bn_relu_conv"):
    with tf.variable_scope(scope, 'bn_relu_conv', [net]):
         with slim.arg_scope([slim.conv2d], padding = 'SAME',
                            activation_fn=tf.nn.relu):
            net = tf.contrib.layers.batch_norm(net)
            net = slim.conv2d(net, n_channels, [3, 3])
            net = slim.dropout(net, drop, is_training=is_training)
            return net

def add_layer(net, n_channels, drop, is_training = True):
    net = bn_relu_conv(net, n_channels, drop, is_training)
    return net

def transition(net, n_channels, drop):
    net = bn_relu_conv(net, n_channels, drop)
    net = slim.max_pool2d(net, [2, 2], stride=[2, 2], padding='SAME')
    return net

def bn_relu_conv_model(input, n_channels, growth_rate, n_classes, levels, drop, is_training):
        
    with slim.arg_scope([slim.conv2d], padding = 'SAME', activation_fn=tf.nn.relu):
        with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.relu):
            net = slim.conv2d(input, n_channels, [3, 3])

            for i in range(levels):
                add_layer(net, n_channels, drop, is_training)
                n_channels += growth_rate
            transition(net, n_channels, drop)

            net = tf.contrib.layers.batch_norm(net)
            net = slim.conv2d(net, n_channels, [3, 3])
            net = slim.flatten(net)
            net = slim.fully_connected(net, n_classes)
            return net

class Model(ModelBase):
    
    feature_depth = 20
    start_channels = 16
    n_classes = 10
    drop = 0.2
    growth_rate = 12
    levels = 10
    
    def zero_pad_features(self, features):
        """ Zero-pad features in the depth dimension to match requested feature depth. """

        feature_pad_size = self.feature_depth - self.num_feature_dimensions
        assert (feature_pad_size >= 0)
        zero_padding = tf.zeros([self.batch_size, 1, self.window_max_points,
                                 feature_pad_size])
        padded = tf.concat(3, [features, zero_padding])

        return padded
    
    def build_training_net(self, features, labels):
        n_channels = self.start_channels

        features = self.zero_pad_features(features)
        one_hot_labels = slim.one_hot_encoding(labels, self.n_classes)

        logits = bn_relu_conv_model(features, n_channels, self.growth_rate, 
                                    self.n_classes, self.levels, self.drop, True)

        loss = slim.losses.softmax_cross_entropy(logits, one_hot_labels)

        optimizer = tf.train.AdadeltaOptimizer(2e-5)

        return TrainNetInfo(loss, optimizer, logits)


    def build_inference_net(self, features):
        n_channels = self.start_channels

        features = self.zero_pad_features(features)

        logits = bn_relu_conv_model(features, n_channels, self.growth_rate, 
                                    self.n_classes, self.levels, self.drop, False)

        return logits